In [ ]:
# Import libraries
import numpy as np
import pandas as pd
import psycopg2
import matplotlib.pyplot as plt
%matplotlib inline
plt.style.use('ggplot') 

In [ ]:
# create a database connection
sqluser = 'postgres'
sqlpass = 'postgres'
dbname = 'mimic_iv'
host = 'localhost'

# Goals
What are we try to do here:
- Provide an example medication/medication administration 
- Pull emar events with pharmacy_id with drug_type BASE, MAIN, ADDITIVE all filled. Generate medication in fhir that rolls up all 3
- Assign the one medication to the 


## Compare mimic tables for best medication source
- prescriptions vs pharmacy vs poe/poe_detail

In [ ]:
con = psycopg2.connect(dbname=dbname, user=sqluser, password=sqlpass, host=host)
cur = con.cursor()

In [ ]:
q_prescriptions = "SELECT DISTINCT drug as medication FROM mimic_hosp.prescriptions;"
q_pharmacy = "SELECT DISTINCT medication FROM mimic_hosp.pharmacy;"
q_poe = """
    SELECT DISTINCT field_value as medication
    FROM
        mimic_hosp.poe poe
        LEFT JOIN mimic_hosp.poe_detail pd
            ON poe.poe_id = pd.poe_id
    WHERE 
        order_type = 'Medications' 
        AND field_value IS NOT NULL;
"""
meds_pharmacy = pd.read_sql_query(q_pharmacy,con)
meds_prescription = pd.read_sql_query(q_prescriptions,con)
meds_poe = pd.read_sql_query(q_poe,con)

In [ ]:
meds_poe.head()

- Based on the values looking at a high level, the poe_detail does not contain individual medication information more on order details.

- Exclude poe_detial, continue with pharmacy and prescriptions

In [ ]:
meds_prescription

In [ ]:
pd.concat([meds_pharmacy,meds_prescription]).drop_duplicates(keep=False)

- There are just over 10,000 medications in both pharmacy and prescription. Comparing for unique values shows that there are 900 medication differences between the two, so worth merging distinct meds here 

In [ ]:
# Updated query
q_meds = """
    SELECT DISTINCT drug as medication FROM mimic_hosp.prescriptions WHERE drug IS NOT NULL
    UNION
    SELECT DISTINCT medication FROM mimic_hosp.pharmacy ph WHERE medication IS NOT NULL
"""
df_meds = pd.read_sql_query(q_meds,con)
df_meds

- BUT looking at emar, which is where medication will be referenced, there are no distinct medications found in the pharmacy that are used by emar

In [ ]:
# Updated query
q_meds_pharmacy = """
    SELECT distinct ph.medication
    FROM 
    mimic_hosp.emar em
    LEFT JOIN mimic_hosp.pharmacy ph
        ON em.pharmacy_id = ph.pharmacy_id
    LEFT JOIN mimic_hosp.prescriptions pr
        ON em.pharmacy_id = pr.pharmacy_id
WHERE
    pr.pharmacy_id IS NULL
    AND ph.pharmacy_id IS NOT NULL
    AND ph.medication NOT IN (SELECT distinct drug as medication 
                              FROM mimic_hosp.prescriptions)
"""
df_meds_pharmacy = pd.read_sql_query(q_meds_pharmacy,con)
df_meds_pharmacy

- Finally that means we just take prescriptions to create the distinct medications

## Create medication resource from prescriptions
1. Create medication resources based on distinct NDC's where NDC != 0
2. Create meciation resources for distinct drugs with NDC=0
    - Need to create an identifier for these drugs, create new schema/table for mapping this
3. We will need to roll up the prescription medications into one FHIR medication resources. These medication mixes will point back to the medication resources generated in step 1 and 2. There are multiple medications that can be under a pharmacy_id, broken down by drug_type being MAIN, BASE or ADDITIVE, assign these each as ingredients 

#### 1. Medication with NDC

In [ ]:
q_medication_ndc = """
WITH vars as (
    SELECT uuid_generate_v5(uuid_generate_v5(uuid_ns_oid(), 'MIMIC-IV'), 'Medication') as uuid_medication
)
SELECT
    pr.ndc as pr_NDC
    , MIN(pr.drug) as pr_DRUG
 
    -- reference uuids
    , uuid_generate_v5(uuid_medication, pr.ndc) as uuid_DRUG
FROM
    mimic_hosp.prescriptions pr
LEFT JOIN vars ON true
WHERE pr.ndc != '0' AND pr.ndc IS NOT NULL AND pr.ndc != '' 
GROUP BY 
    pr.ndc
    , uuid_medication
LIMIT 10
"""
df_meds_ndc = pd.read_sql_query(q_medication_ndc,con)
df_meds_ndc

#### 2. Medication with no NDC

In [ ]:
# Create new schema and identifier for drugs with no NDC
q_map_drug_id = """
    DROP TABLE IF EXISTS fhir_etl.map_drug_id;
    CREATE TABLE fhir_etl.map_drug_id(
        drug_id INT GENERATED ALWAYS AS IDENTITY,
        drug VARCHAR NOT NULL
    );

    INSERT INTO fhir_etl.map_drug_id(drug)
    SELECT DISTINCT drug 
    FROM mimic_hosp.prescriptions
    WHERE ndc = '0' or ndc IS NULL OR ndc = '';
"""
cur.execute(q_medication_other)

In [ ]:
q_medication_other = """
WITH vars as (
    SELECT uuid_generate_v5(uuid_generate_v5(uuid_ns_oid(), 'MIMIC-IV'), 'Medication') as uuid_medication
)
SELECT
    DISTINCT pr.drug as pr_DRUG
    , pr.ndc as pr_NDC
    , md.drug_id::text as md_DRUG_ID
  
    -- reference uuids
    , uuid_generate_v5(uuid_medication, md.drug_id::text) as uuid_DRUG
FROM
    mimic_hosp.prescriptions pr	
    LEFT JOIN fhir_etl.map_drug_id md
        ON pr.drug = md.drug
    LEFT JOIN vars ON true
WHERE pr.ndc = '0' OR pr.ndc IS NULL OR pr.ndc = ''
LIMIT 10
"""
df_meds_other = pd.read_sql_query(q_medication_other,con)
df_meds_other

#### 3. Medication mixes

In [ ]:
q_medication_mix = """
WITH vars as (
    SELECT uuid_generate_v5(uuid_generate_v5(uuid_ns_oid(), 'MIMIC-IV'), 'Medication') as uuid_medication
)
SELECT
    pr.pharmacy_id as pr_PHARMACY_ID

  
    , json_agg(json_build_object(
        'itemReference', 
            jsonb_build_object('reference', 'Medication/' || 
                               uuid_generate_v5(uuid_medication,CASE 
                                                                WHEN md.drug_id IS NOT NULL 
                                                                THEN md.drug_id::text 
                                                                ELSE pr.ndc::text END)                

                                )
    )) as pr_INGREDIENTS
  
    -- reference uuids
    , uuid_generate_v5(uuid_medication, pr.pharmacy_id::text) as uuid_DRUG
FROM
    mimic_hosp.prescriptions pr
    LEFT JOIN fhir_etl.map_drug_id md
        ON pr.drug = md.drug
    LEFT JOIN vars ON true
GROUP BY 
    pr.pharmacy_id
    , uuid_medication
LIMIT 10

"""
df_meds_mix = pd.read_sql_query(q_medication_mix,con)
df_meds_mix

## Create medication administration resource from prescriptions
- With medication mixes being assigned in the Medication resource, medication administration fairly straight forward. Assign single FHIR medication to medication administration based on reference UUID.

## Create medication request resource from pharamcy
- Pharmacy only has one medication per pharmacy_id vs prescription which allows multiple medications. The pharmacy table just grabs the main/base drug from prescription. So we can use the pharmacy table for the medication request

In [ ]:
q_pharmacy_multi_med = """
SELECT pharmacy_id
FROM mimic_hosp.pharmacy
GROUP BY pharmacy_id
HAVING COUNT(medication) > 2
"""
df_pharmacy_multi_med= pd.read_sql_query(q_pharmacy_multi_med,con)

In [ ]:
q_prescription_multi_med = """
SELECT pharmacy_id
FROM mimic_hosp.prescriptions
GROUP BY pharmacy_id
HAVING COUNT(drug) > 2
"""
df_prescription_multi_med= pd.read_sql_query(q_prescription_multi_med,con)

In [ ]:
df_pharmacy_multi_med

In [ ]:
df_prescription_multi_med